In [3]:
%pip install --quiet moviepy pandas numpy shapely geopy plotly tqdm rtree matplotlib


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import List, Dict, Optional, Tuple, NamedTuple

In [2]:
data_dir = Path("./data")

flight_log_path = data_dir / "flight_log.csv"
tracking_paths = [data_dir / "tracking1.json", data_dir / "tracking2.json"]

In [3]:
# 기본 데이터 모델
class Point(NamedTuple):
    x: float
    y: float


class GeoPoint(NamedTuple):
    latitude: float
    longitude: float


@dataclass
class DroneState:
    position: GeoPoint
    altitude: float
    heading: float
    timestamp: float

In [4]:
def find_video_segments(flight_log: pd.DataFrame) -> List[pd.DataFrame]:
    """비디오 구간을 찾아서 반환"""
    segments = []
    current_segment = []

    for _, row in flight_log.iterrows():
        if row["isVideo"] == 1:
            current_segment.append(row)
        else:
            if current_segment:
                segments.append(pd.DataFrame(current_segment))
                current_segment = []

    if current_segment:
        segments.append(pd.DataFrame(current_segment))

    return segments

In [5]:
# 실행
flight_log = pd.read_csv(flight_log_path)
segments = find_video_segments(flight_log)

print(f"총 {len(segments)}개의 비디오 구간을 찾았습니다.\n")
for i, segment in enumerate(segments, 1):
    start_time = segment["time(millisecond)"].iloc[0] / 1000
    end_time = segment["time(millisecond)"].iloc[-1] / 1000
    duration = end_time - start_time

    print(f"구간 {i}:")
    print(f"  시작 시간: {start_time:.2f}초")
    print(f"  종료 시간: {end_time:.2f}초")
    print(f"  지속 시간: {duration:.2f}초")
    print(f"  데이터 포인트 수: {len(segment)}")
    print(
        f"  시작 위치: ({segment['latitude'].iloc[0]:.6f}, {segment['longitude'].iloc[0]:.6f})"
    )
    print(
        f"  종료 위치: ({segment['latitude'].iloc[-1]:.6f}, {segment['longitude'].iloc[-1]:.6f})"
    )
    print(f"  로깅 간격: {(duration/len(segment)):.3f}초")
    print()

총 2개의 비디오 구간을 찾았습니다.

구간 1:
  시작 시간: 591.60초
  종료 시간: 899.20초
  지속 시간: 307.60초
  데이터 포인트 수: 1539
  시작 위치: (37.451060, 126.656443)
  종료 위치: (37.451059, 126.656434)
  로깅 간격: 0.200초

구간 2:
  시작 시간: 967.40초
  종료 시간: 1217.60초
  지속 시간: 250.20초
  데이터 포인트 수: 1251
  시작 위치: (37.449199, 126.656013)
  종료 위치: (37.449199, 126.656008)
  로깅 간격: 0.200초



In [6]:
def create_frame_drone_mapping(
    segment_df: pd.DataFrame, tracking_data: dict
) -> List[Dict]:
    """프레임별 드론 상태 매핑"""
    fps = tracking_data["video"]["fps"]
    segment_start_time = segment_df["time(millisecond)"].iloc[0] / 1000

    frames = []
    for frame_data in tqdm(tracking_data["tracking_results"], desc="프레임 매핑"):
        frame_idx = frame_data["i"]
        frame_time = segment_start_time + (frame_idx / fps)

        # 가장 가까운 flight log 찾기
        closest_log = segment_df.iloc[
            (segment_df["time(millisecond)"] / 1000 - frame_time).abs().argmin()
        ]

        frame_info = {
            "frame_index": frame_idx,
            "timestamp": frame_time,
            "drone_state": {
                "latitude": closest_log["latitude"],
                "longitude": closest_log["longitude"],
                "altitude": closest_log["ascent(feet)"] * 0.3048,  # feet to meters
                "heading": closest_log["compass_heading(degrees)"],
            },
            "objects": frame_data["res"],
        }
        frames.append(frame_info)

    return frames

In [7]:
# 첫 번째 세그먼트에 대해 실행
with open(tracking_paths[0], "r") as f:
    tracking_data = json.load(f)

frames = create_frame_drone_mapping(segments[0], tracking_data)

# 첫 다섯개 프레임 정보 출력
for i in range(5):
    print(f"첫 {i} 프레임 정보:")
    print(f"프레임 인덱스: {frames[i]['frame_index']}")
    print(f"타임스탬프: {frames[i]['timestamp']:.2f}초")
    print(f"드론 상태:")
    print(f"  위도: {frames[i]['drone_state']['latitude']}")
    print(f"  경도: {frames[i]['drone_state']['longitude']}")
    print(f"  고도: {frames[i]['drone_state']['altitude']:.1f}m")
    print(f"  방향: {frames[i]['drone_state']['heading']}°")
    print(f"객체 수: {len(frames[i]['objects'])}")

프레임 매핑: 100%|██████████| 9223/9223 [00:00<00:00, 10903.05it/s]

첫 0 프레임 정보:
프레임 인덱스: 0
타임스탬프: 591.60초
드론 상태:
  위도: 37.4510599991108
  경도: 126.656443306919
  고도: 40.0m
  방향: 289.1°
객체 수: 68
첫 1 프레임 정보:
프레임 인덱스: 1
타임스탬프: 591.63초
드론 상태:
  위도: 37.4510599991108
  경도: 126.656443306919
  고도: 40.0m
  방향: 289.1°
객체 수: 68
첫 2 프레임 정보:
프레임 인덱스: 2
타임스탬프: 591.67초
드론 상태:
  위도: 37.4510599991108
  경도: 126.656443306919
  고도: 40.0m
  방향: 289.1°
객체 수: 68
첫 3 프레임 정보:
프레임 인덱스: 3
타임스탬프: 591.70초
드론 상태:
  위도: 37.4510601350421
  경도: 126.656443250977
  고도: 40.0m
  방향: 289.1°
객체 수: 69
첫 4 프레임 정보:
프레임 인덱스: 4
타임스탬프: 591.73초
드론 상태:
  위도: 37.4510601350421
  경도: 126.656443250977
  고도: 40.0m
  방향: 289.1°
객체 수: 69


In [8]:
from dataclasses import dataclass
from typing import List, Dict, Tuple
from rtree import index
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm


@dataclass
class Segment:
    """선분 정보를 담는 데이터 클래스"""

    track_id: int
    p1: Tuple[float, float]  # (x1, y1)
    p2: Tuple[float, float]  # (x2, y2)
    t1: float  # timestamp1
    t2: float  # timestamp2
    frame1: int
    frame2: int


def get_trajectory_segments(
    frames: List[Dict], window_start: int, window_size: int
) -> List[Segment]:
    """프레임 윈도우에서 모든 궤적 선분 추출"""
    segments = []
    window_frames = frames[window_start : window_start + window_size]

    object_positions = {}
    for frame in window_frames:
        for obj in frame["objects"]:
            track_id = obj["tid"]
            if track_id not in object_positions:
                object_positions[track_id] = []

            # 중심점 계산
            x = (obj["bbox"][0] + obj["bbox"][2]) / 2
            y = (obj["bbox"][1] + obj["bbox"][3]) / 2

            object_positions[track_id].append(
                {
                    "point": (x, y),
                    "frame": frame["frame_index"],
                    "time": frame["timestamp"],
                }
            )

    # 선분 생성
    for track_id, positions in object_positions.items():
        if len(positions) < 2:
            continue
        for i in range(len(positions) - 1):
            segments.append(
                Segment(
                    track_id=track_id,
                    p1=positions[i]["point"],
                    p2=positions[i + 1]["point"],
                    t1=positions[i]["time"],
                    t2=positions[i + 1]["time"],
                    frame1=positions[i]["frame"],
                    frame2=positions[i + 1]["frame"],
                )
            )

    return segments


def line_segments_intersect(seg1: Segment, seg2: Segment) -> Tuple[float, float]:
    """두 선분의 교차점 계산"""
    x1, y1 = seg1.p1
    x2, y2 = seg1.p2
    x3, y3 = seg2.p1
    x4, y4 = seg2.p2

    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if denominator == 0:
        return None

    t = ((x1 - x3) * (y3 - y4) - (y1 - y3) * (x3 - x4)) / denominator
    u = -((x1 - x2) * (y1 - y3) - (y1 - y2) * (x1 - x3)) / denominator

    if 0 <= t <= 1 and 0 <= u <= 1:
        x = x1 + t * (x2 - x1)
        y = y1 + t * (y2 - y1)
        return (x, y)

    return None


def process_window(args) -> List[Dict]:
    """하나의 윈도우 처리"""
    frames, window_start, window_size = args
    segments = get_trajectory_segments(frames, window_start, window_size)

    # R-Tree 인덱스 생성
    idx = index.Index()
    for i, seg in enumerate(segments):
        # 선분의 bounding box 계산
        bbox = (
            min(seg.p1[0], seg.p2[0]),
            min(seg.p1[1], seg.p2[1]),
            max(seg.p1[0], seg.p2[0]),
            max(seg.p1[1], seg.p2[1]),
        )
        idx.insert(i, bbox)  # 각 선분을 개별 삽입

    # 교차점 계산
    intersections = []
    processed_pairs = set()

    for i, seg1 in enumerate(segments):
        bbox = (
            min(seg1.p1[0], seg1.p2[0]),
            min(seg1.p1[1], seg1.p2[1]),
            max(seg1.p1[0], seg1.p2[0]),
            max(seg1.p1[1], seg1.p2[1]),
        )
        for j in idx.intersection(bbox):
            if i >= j:  # 중복 방지
                continue
            seg2 = segments[j]
            pair = tuple(sorted([seg1.track_id, seg2.track_id]))
            if pair in processed_pairs:
                continue
            processed_pairs.add(pair)
            if seg1.track_id == seg2.track_id:
                continue

            intersection = line_segments_intersect(seg1, seg2)
            if intersection:
                time_diff = abs(seg1.t1 - seg2.t1)
                intersections.append(
                    {
                        "frame_index": min(seg1.frame1, seg2.frame1),
                        "timestamp": min(seg1.t1, seg2.t1),
                        "track_id1": seg1.track_id,
                        "track_id2": seg2.track_id,
                        "intersection_point": intersection,
                        "time_difference": time_diff,
                    }
                )

    return intersections


def analyze_trajectories_in_frame_window(
    frames: List[Dict], window_size: int = 30 * 10
) -> List[Dict]:
    """멀티스레딩을 사용한 궤적 분석"""
    tasks = [
        (frames, i, window_size)
        for i in range(0, len(frames) - window_size, window_size)
    ]

    # 멀티스레딩 실행
    all_intersections = []
    with ThreadPoolExecutor() as executor:
        for result in tqdm(
            executor.map(process_window, tasks), total=len(tasks), desc="궤적 분석"
        ):
            all_intersections.extend(result)

    return all_intersections

In [9]:
# 픽셀 속도 계싼해봅시다.
import math


def calculate_pixel_speed(frames):
    """
    각 객체의 픽셀 속도를 계산합니다.
    Args:
        frames (List[Dict]): 프레임 정보
    Returns:
        Dict: 객체별 픽셀 속도 정보
    """
    object_speeds = {}

    for frame in frames:
        for obj in frame["objects"]:
            track_id = obj["tid"]
            # 중심점 계산
            x = (obj["bbox"][0] + obj["bbox"][2]) / 2
            y = (obj["bbox"][1] + obj["bbox"][3]) / 2
            timestamp = frame["timestamp"]

            if track_id not in object_speeds:
                # 객체 초기화
                object_speeds[track_id] = {"positions": [], "speeds": []}

            # 현재 위치 저장
            object_speeds[track_id]["positions"].append(
                {"x": x, "y": y, "timestamp": timestamp}
            )

    # 속도 계산
    for track_id, data in object_speeds.items():
        positions = data["positions"]
        for i in range(1, len(positions)):
            prev = positions[i - 1]
            curr = positions[i]
            # 픽셀 거리 계산
            distance = math.sqrt(
                (curr["x"] - prev["x"]) ** 2 + (curr["y"] - prev["y"]) ** 2
            )
            # 시간 차이 계산
            time_diff = curr["timestamp"] - prev["timestamp"]
            # 속도 계산 (pixels/second)
            if time_diff > 0:
                speed = distance / time_diff
                data["speeds"].append(speed)
            else:
                data["speeds"].append(0)

    return object_speeds

In [10]:
# 예제 실행
object_pixel_speeds = calculate_pixel_speed(frames)

# 객체별 평균 픽셀 속도 출력
for track_id, data in object_pixel_speeds.items():
    average_speed = sum(data["speeds"]) / len(data["speeds"]) if data["speeds"] else 0
    print(f"객체 {track_id}의 평균 픽셀 속도: {average_speed:.2f} pixels/second")

객체 1의 평균 픽셀 속도: 9.22 pixels/second
객체 2의 평균 픽셀 속도: 6.68 pixels/second
객체 3의 평균 픽셀 속도: 12.56 pixels/second
객체 4의 평균 픽셀 속도: 6.33 pixels/second
객체 5의 평균 픽셀 속도: 4.15 pixels/second
객체 6의 평균 픽셀 속도: 58.89 pixels/second
객체 7의 평균 픽셀 속도: 287.81 pixels/second
객체 8의 평균 픽셀 속도: 4.83 pixels/second
객체 9의 평균 픽셀 속도: 11.07 pixels/second
객체 10의 평균 픽셀 속도: 75.28 pixels/second
객체 11의 평균 픽셀 속도: 70.69 pixels/second
객체 12의 평균 픽셀 속도: 61.78 pixels/second
객체 13의 평균 픽셀 속도: 65.59 pixels/second
객체 14의 평균 픽셀 속도: 52.24 pixels/second
객체 15의 평균 픽셀 속도: 92.79 pixels/second
객체 16의 평균 픽셀 속도: 38.85 pixels/second
객체 17의 평균 픽셀 속도: 8.56 pixels/second
객체 18의 평균 픽셀 속도: 28.03 pixels/second
객체 19의 평균 픽셀 속도: 72.13 pixels/second
객체 20의 평균 픽셀 속도: 67.01 pixels/second
객체 21의 평균 픽셀 속도: 27.00 pixels/second
객체 22의 평균 픽셀 속도: 73.56 pixels/second
객체 23의 평균 픽셀 속도: 69.33 pixels/second
객체 24의 평균 픽셀 속도: 95.71 pixels/second
객체 25의 평균 픽셀 속도: 75.40 pixels/second
객체 26의 평균 픽셀 속도: 72.03 pixels/second
객체 27의 평균 픽셀 속도: 64.70 pixels/second
객체 28의 평균 픽셀 속도

In [11]:
import math


class GeoreferencingService:
    def __init__(self, camera_fov, video_width, video_height):
        """
        Georeferencing 서비스 초기화.

        Args:
            camera_fov (float): 카메라의 수평 시야각 (degrees)
            video_width (int): 비디오 해상도 가로 크기 (pixels)
            video_height (int): 비디오 해상도 세로 크기 (pixels)
        """
        if not (0 < camera_fov < 180):
            raise ValueError("카메라 수평 시야각은 0~180도 사이여야 합니다.")
        if video_width <= 0 or video_height <= 0:
            raise ValueError("비디오 해상도가 유효하지 않습니다.")

        self.camera_fov = camera_fov
        self.video_width = video_width
        self.video_height = video_height
        self.center_x = video_width / 2
        self.center_y = video_height / 2
        self.diagonal_pixels = math.sqrt(video_width**2 + video_height**2)

    def normalize_coordinates(self, bbox):
        """
        객체의 중심점을 정규화된 좌표로 변환 (-1 to 1 범위).

        Args:
            bbox (list): [x1, y1, x2, y2] 바운딩 박스 좌표.

        Returns:
            dict: 정규화된 x, y 좌표.
        """
        x1, y1, x2, y2 = bbox
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2

        return {
            "x": (center_x - self.center_x) / self.center_x,
            "y": (center_y - self.center_y) / self.center_y,
        }

    def calculate_object_location(self, bbox, drone_position, heading, altitude):
        """
        객체의 지리적 위치 계산.

        Args:
            bbox (list): [x1, y1, x2, y2] 바운딩 박스 좌표.
            drone_position (tuple): 드론의 GPS 위치 (latitude, longitude).
            heading (float): 드론의 진행 방향 (degrees).
            altitude (float): 드론의 고도 (meters).

        Returns:
            tuple: 객체의 GPS 위치 (latitude, longitude).
        """
        normalized = self.normalize_coordinates(bbox)

        # 영상 중심으로부터의 상대적 거리 계산
        pixel_distance = math.sqrt(
            (normalized["x"] * self.center_x) ** 2
            + (normalized["y"] * self.center_y) ** 2
        )

        # 실제 거리 계산
        distance_ratio = pixel_distance / self.diagonal_pixels
        fov_radians = math.radians(self.camera_fov)
        ground_distance = altitude * math.tan(fov_radians / 2) * distance_ratio

        # 방위각 계산 (y축 반전)
        bearing = math.degrees(math.atan2(normalized["x"], -normalized["y"]))
        bearing = (bearing + heading) % 360

        # 최종 위치 계산
        return self.calculate_destination(drone_position, ground_distance, bearing)

    def calculate_destination(self, start_coords, distance, bearing):
        """
        시작 좌표에서 주어진 거리와 방위각으로 이동한 결과 좌표 계산.

        Args:
            start_coords (tuple): 시작 GPS 좌표 (latitude, longitude).
            distance (float): 이동 거리 (meters).
            bearing (float): 방위각 (degrees).

        Returns:
            tuple: 이동한 결과 GPS 좌표 (latitude, longitude).
        """
        lat1 = math.radians(start_coords[0])
        lon1 = math.radians(start_coords[1])
        bearing_rad = math.radians(bearing)
        earth_radius = 6371000  # 지구 반지름 (meters)

        # 새로운 좌표 계산
        lat2 = math.asin(
            math.sin(lat1) * math.cos(distance / earth_radius)
            + math.cos(lat1) * math.sin(distance / earth_radius) * math.cos(bearing_rad)
        )
        lon2 = lon1 + math.atan2(
            math.sin(bearing_rad) * math.sin(distance / earth_radius) * math.cos(lat1),
            math.cos(distance / earth_radius) - math.sin(lat1) * math.sin(lat2),
        )

        return math.degrees(lat2), math.degrees(lon2)

In [12]:
# DJI Mini 2의 FOV
DJI_MINI2_FOV = 83  # 수평 시야각 (degrees)

for i, segment in enumerate(segments):
    print(f"Processing video segment {i + 1}/{len(segments)}...")

    # Tracking 데이터 로드
    with open(tracking_paths[i], "r") as f:
        tracking_data = json.load(f)

    # 비디오 메타데이터에서 이미지 크기 가져오기
    video_metadata = tracking_data["video"]
    georef_service = GeoreferencingService(
        camera_fov=DJI_MINI2_FOV,
        video_width=video_metadata["width"],
        video_height=video_metadata["height"],
    )

    # 프레임 데이터 생성
    frames = create_frame_drone_mapping(segment, tracking_data)

    # 픽셀 속도 계산
    object_pixel_speeds = calculate_pixel_speed(frames)

    # 픽셀 크기 변환 비율 계산
    fov_radians = math.radians(DJI_MINI2_FOV)
    altitude = frames[0]["drone_state"]["altitude"]  # 첫 프레임의 고도 사용
    ground_width = 2 * altitude * math.tan(fov_radians / 2)  # 지면 상의 폭 (meters)
    meters_per_pixel = (
        ground_width / video_metadata["width"]
    )  # 픽셀당 실제 거리 (meters)

    # 첫 두 프레임 출력
    for j in range(2):
        print(f"\n비디오 {i + 1}, 첫 {j + 1} 프레임 정보:")
        print(f"프레임 인덱스: {frames[j]['frame_index']}")
        print(f"타임스탬프: {frames[j]['timestamp']:.2f}초")
        print(f"드론 상태:")
        print(f"  위도: {frames[j]['drone_state']['latitude']}")
        print(f"  경도: {frames[j]['drone_state']['longitude']}")
        print(f"  고도: {frames[j]['drone_state']['altitude']:.1f}m")
        print(f"  방향: {frames[j]['drone_state']['heading']}°")
        print(f"객체 수: {len(frames[j]['objects'])}")

    # 교차점 데이터 생성
    intersections = analyze_trajectories_in_frame_window(frames)
    print(f"\n발견된 교차점 수: {len(intersections)}")
    if intersections:
        print("\n첫 5개 교차점:")
        for k, intersection in enumerate(
            sorted(intersections, key=lambda x: -x["time_difference"])[:5]
        ):
            print(f"\n교차점 {k + 1}:")
            print(intersection)

    # 계층적 JSON 구조 생성
    video_data = {
        "start_time": segment["time(millisecond)"].iloc[0] / 1000,
        "end_time": segment["time(millisecond)"].iloc[-1] / 1000,
        "start_location": {
            "latitude": segment["latitude"].iloc[0],
            "longitude": segment["longitude"].iloc[0],
        },
        "end_location": {
            "latitude": segment["latitude"].iloc[-1],
            "longitude": segment["longitude"].iloc[-1],
        },
        "frames": [],
        "tracking": [],
        "intersections": [],
    }

    # 프레임 추가
    for frame in frames:
        frame_data = {
            "frame_index": frame["frame_index"],
            "timestamp": frame["timestamp"],
            "drone_state": frame["drone_state"],
            "detections": [],
        }

        # Detection 처리
        for obj in frame["objects"]:
            # 바운딩 박스 중심의 GPS 계산
            gps_coords = georef_service.calculate_object_location(
                bbox=obj["bbox"],
                drone_position=(
                    frame["drone_state"]["latitude"],
                    frame["drone_state"]["longitude"],
                ),
                heading=frame["drone_state"]["heading"],
                altitude=frame["drone_state"]["altitude"],
            )

            # 월드 속도 계산
            pixel_speed = (
                object_pixel_speeds[obj["tid"]]["speeds"][0]
                if obj["tid"] in object_pixel_speeds
                and object_pixel_speeds[obj["tid"]]["speeds"]
                else 0
            )
            world_speed = pixel_speed * meters_per_pixel

            # Detection 데이터에 월드 속도와 GPS 좌표 추가
            frame_data["detections"].append(
                {
                    "class_id": obj["cid"],
                    "track_id": obj["tid"],
                    "bbox": obj["bbox"],
                    "confidence": obj["conf"],
                    "world_speed": world_speed,
                    "gps_coordinates": {
                        "latitude": gps_coords[0],
                        "longitude": gps_coords[1],
                    },
                }
            )

        video_data["frames"].append(frame_data)

    # 트래킹 데이터 추가
    trajectory_segments = get_trajectory_segments(frames, 0, len(frames))
    for segment in trajectory_segments:
        video_data["tracking"].append(
            {
                "track_id": segment.track_id,
                "frame_start": segment.frame1,
                "frame_end": segment.frame2,
                "start_point": segment.p1,
                "end_point": segment.p2,
                "timestamp_start": segment.t1,
                "timestamp_end": segment.t2,
            }
        )

    # 교차점 추가
    for intersection in intersections:
        intersection_gps_coords = georef_service.calculate_object_location(
            bbox=[
                intersection["intersection_point"][0],
                intersection["intersection_point"][1],
                intersection["intersection_point"][0],
                intersection["intersection_point"][1],
            ],
            drone_position=(
                frame["drone_state"]["latitude"],
                frame["drone_state"]["longitude"],
            ),
            heading=frame["drone_state"]["heading"],
            altitude=frame["drone_state"]["altitude"],
        )
        video_data["intersections"].append(
            {
                "frame_index": intersection["frame_index"],
                "timestamp": intersection["timestamp"],
                "track_id1": intersection["track_id1"],
                "track_id2": intersection["track_id2"],
                "intersection_point": intersection["intersection_point"],
                "gps_coordinates": {
                    "latitude": intersection_gps_coords[0],
                    "longitude": intersection_gps_coords[1],
                },
                "time_difference": intersection["time_difference"],
            }
        )

    # JSON 파일로 저장
    output_file = f"segment_{i + 1}.json"
    with open(output_file, "w") as f:
        json.dump(video_data, f, indent=2)

    print(f"비디오 세그먼트 {i + 1} JSON 파일로 저장되었습니다: {output_file}")

Processing video segment 1/2...


프레임 매핑: 100%|██████████| 9223/9223 [00:01<00:00, 7701.63it/s] 



비디오 1, 첫 1 프레임 정보:
프레임 인덱스: 0
타임스탬프: 591.60초
드론 상태:
  위도: 37.4510599991108
  경도: 126.656443306919
  고도: 40.0m
  방향: 289.1°
객체 수: 68

비디오 1, 첫 2 프레임 정보:
프레임 인덱스: 1
타임스탬프: 591.63초
드론 상태:
  위도: 37.4510599991108
  경도: 126.656443306919
  고도: 40.0m
  방향: 289.1°
객체 수: 68


궤적 분석: 100%|██████████| 30/30 [00:34<00:00,  1.16s/it]



발견된 교차점 수: 865

첫 5개 교차점:

교차점 1:
{'frame_index': 2100, 'timestamp': np.float64(661.6700000000001), 'track_id1': 1033, 'track_id2': 1203, 'intersection_point': (3798.8519711862505, 1779.6623222335807), 'time_difference': np.float64(9.94326666666666)}

교차점 2:
{'frame_index': 4500, 'timestamp': np.float64(741.75), 'track_id1': 58, 'track_id2': 2065, 'intersection_point': (646.1100661718681, 460.1118117490573), 'time_difference': np.float64(9.909899999999993)}

교차점 3:
{'frame_index': 6001, 'timestamp': np.float64(791.8333666666667), 'track_id1': 2154, 'track_id2': 2400, 'intersection_point': (2773.4315012389684, 1174.6255821522943), 'time_difference': np.float64(9.876533333333327)}

교차점 4:
{'frame_index': 604, 'timestamp': np.float64(611.7534666666667), 'track_id1': 247, 'track_id2': 312, 'intersection_point': (3770.6862051170056, 846.053899180088), 'time_difference': np.float64(9.476133333333337)}

교차점 5:
{'frame_index': 3610, 'timestamp': np.float64(712.0536666666667), 'track_id1': 134

프레임 매핑: 100%|██████████| 7498/7498 [00:01<00:00, 7142.36it/s] 



비디오 2, 첫 1 프레임 정보:
프레임 인덱스: 0
타임스탬프: 967.40초
드론 상태:
  위도: 37.4491993632329
  경도: 126.656012908395
  고도: 40.3m
  방향: 268.1°
객체 수: 9

비디오 2, 첫 2 프레임 정보:
프레임 인덱스: 1
타임스탬프: 967.43초
드론 상태:
  위도: 37.4491993632329
  경도: 126.656012908395
  고도: 40.3m
  방향: 268.1°
객체 수: 9


궤적 분석: 100%|██████████| 24/24 [00:06<00:00,  3.67it/s]



발견된 교차점 수: 73

첫 5개 교차점:

교차점 1:
{'frame_index': 2406, 'timestamp': np.float64(1047.6802), 'track_id1': 207, 'track_id2': 288, 'intersection_point': (1016.7924876709712, 1565.404410301429), 'time_difference': np.float64(8.07473333333337)}

교차점 2:
{'frame_index': 5434, 'timestamp': np.float64(1148.7144666666666), 'track_id1': 592, 'track_id2': 665, 'intersection_point': (2488.7267467163224, 1947.6698973228017), 'time_difference': np.float64(7.941266666666706)}

교차점 3:
{'frame_index': 3914, 'timestamp': np.float64(1097.9971333333333), 'track_id1': 444, 'track_id2': 467, 'intersection_point': (1338.4561928278342, 1410.0165066409972), 'time_difference': np.float64(7.540866666666716)}

교차점 4:
{'frame_index': 6633, 'timestamp': np.float64(1188.7211), 'track_id1': 758, 'track_id2': 784, 'intersection_point': (2049.473588185539, 1194.1669040341553), 'time_difference': np.float64(7.507499999999936)}

교차점 5:
{'frame_index': 4506, 'timestamp': np.float64(1117.7502), 'track_id1': 499, 'track_id2'